In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/migros-datathon-coderspace/customeraccount.csv
/kaggle/input/migros-datathon-coderspace/sample_submission.csv
/kaggle/input/migros-datathon-coderspace/transaction_header.csv
/kaggle/input/migros-datathon-coderspace/genel_kategoriler.csv
/kaggle/input/migros-datathon-coderspace/customer.csv
/kaggle/input/migros-datathon-coderspace/train.csv
/kaggle/input/migros-datathon-coderspace/test.csv
/kaggle/input/migros-datathon-coderspace/product_groups.csv
/kaggle/input/migros-datathon-coderspace/transaction_sale/transaction_sale.csv


In [13]:
%pip install nb_black

import warnings
from tqdm import tqdm
from pathlib import Path
from itertools import repeat

import numpy as np
import pandas as pd
from scipy.stats import skew

from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold
from sklearn.model_selection import train_test_split, cross_validate


from lightgbm import LGBMRegressor
import lightgbm as lgbm
from catboost import CatBoostClassifier, Pool
from catboost.utils import get_roc_curve, select_threshold

%load_ext lab_black
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: "%.3f" % x)
pd.set_option("display.width", 500)

Note: you may need to restart the kernel to use updated packages.
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [99]:
# submission_csv = pd.read_csv('../input/2022gdz22-datathon-case-2/submission.csv')
test_ = pd.read_csv("/kaggle/input/migros-datathon-coderspace/test.csv")
test = test_.copy()
train_ = pd.read_csv("/kaggle/input/migros-datathon-coderspace/train.csv")
train = train_.copy()


customer = pd.read_csv("/kaggle/input/migros-datathon-coderspace/customer.csv")
customeraccount = pd.read_csv(
    "/kaggle/input/migros-datathon-coderspace/customeraccount.csv"
)
genel_kategoriler = pd.read_csv(
    "/kaggle/input/migros-datathon-coderspace/genel_kategoriler.csv"
)
product_groups = pd.read_csv(
    "/kaggle/input/migros-datathon-coderspace/product_groups.csv"
)
transaction_header = pd.read_csv(
    "/kaggle/input/migros-datathon-coderspace/transaction_header.csv"
)
transaction_sale = pd.read_csv(
    "/kaggle/input/migros-datathon-coderspace/transaction_sale/transaction_sale.csv"
)

In [103]:
test.nunique()

individualnumber    13236
category_number        49
hakkedis_amt          118
odul_amt               20
dtype: int64

In [115]:
train.nunique()

individualnumber    13115
category_number        49
hakkedis_amt          121
odul_amt               20
response                2
dtype: int64

In [114]:
customeraccount.nunique()

individualnumber    28593
cardnumber          35159
dtype: int64

In [112]:
customer.min()

individualnumber   30690.000
city_code              1.000
dateofbirth         1920.000
dtype: float64

In [100]:
customer.head()

,individualnumber,gender,city_code,dateofbirth
0,94212124,K,NaN,1946.000
1,96387515,E,34.000,1974.000
2,95040383,E,80.000,1980.000
3,94694434,E,NaN,1970.000
4,47648671,E,35.000,1963.000


In [15]:
train.head()

,individualnumber,category_number,hakkedis_amt,odul_amt,response
0,94230288,9000,21.000,2.000,0
1,4684087,9000,17.000,1.000,0
2,92472145,9058,24.000,3.000,0
3,88026681,9030,22.000,2.000,0
4,98127795,9001,38.000,3.000,0


In [121]:
genel_kategoriler.max()

category_number             9061
genel_kategori     kisisel_bakim
dtype: object

In [127]:
product_groups.head()

,category_number,category_level_1,category_level_2,category_level_3,category_level_4
0,9046,341,20,10,10
1,9009,125,25,10,10
2,9049,410,65,20,10
3,9019,300,10,19,11
4,9035,305,31,10,10


In [126]:
transaction_header.head()

,date_of_transaction,cardnumber,basketid,is_sanal
0,2020-12-01,8487374630565615,20120163990041163,0
1,2020-12-01,4287574603569015,20120163990016137,0
2,2020-12-01,6487154648560315,20120160080017338,0
3,2020-12-01,3787564693563915,20120163570105022,0
4,2020-12-01,9087994683569715,20120162340016645,0


In [125]:
transaction_sale.head()

,basketid,category_level_1,category_level_2,category_level_3,category_level_4,amount,quantity,discount_type_1,discount_type_2,discount_type_3
0,20120706070177471,613,50,50,15,19.900,2.000,NaN,0.000,0.000
1,20120706070177471,425,30,12,10,3.500,1.000,NaN,0.000,0.000
2,20120706070177471,425,30,12,11,7.900,1.000,NaN,0.000,0.000
3,20122703940031323,103,25,15,10,7.900,2.000,NaN,0.000,0.000
4,20122703940031323,101,15,15,15,16.000,1.000,NaN,0.000,0.000


In [16]:
test.head()

,individualnumber,category_number,hakkedis_amt,odul_amt
0,95812936,9029,38.000,3.000
1,23929570,9012,28.000,2.000
2,95948115,9019,21.000,2.000
3,100898513,9035,48.000,4.000
4,12769156,9015,26.000,3.000


In [17]:
print("train obs. points:", train.shape[0])
print("test obs points:", test.shape[0])
test.shape[0] / (train.shape[0] + test.shape[0])

train obs. points: 13115
test obs points: 13236


0.5022959280482714

In [18]:
# Column that existing in train but not in test.
[col for col in train.columns if col not in test.columns]

['response']

In [19]:
# Column that is existing in test but not in train.
[col for col in test.columns if col not in train.columns]

[]

In [20]:
y = train["response"]
X = train.drop(["response"], axis=1)
X_test = test

In [22]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.25, random_state=17
)  # 0.25 x 0.8 = 0.2

In [24]:
datasets = {
    "x_train": X_train,
    "y_train": y_train,
    "x_val": X_val,
    "y_val": y_val,
    "x_test": X_test,
}
# X_train, X_test, y_train, y_test
def pred_model(model, **datasets):
    pred_val = model.predict(datasets["x_val"])
    pred_val = pd.DataFrame(pred_val, index=datasets["x_val"].index, columns=["pred"])

    pred_test = model.predict(datasets["x_test"])

    return pred_test, pred_val

In [46]:
X_train.isnull().sum()

individualnumber    0
category_number     0
hakkedis_amt        0
odul_amt            0
dtype: int64

In [38]:
lgbm_params = {
    "learning_rate": [0.01, 0.03],
    "colsample_bytree": [0.3, 0.7],
    "num_boost_round": [500],
}

In [51]:
X.head()

,individualnumber,category_number,hakkedis_amt,odul_amt
0,94230288,9000,21.000,2.000
1,4684087,9000,17.000,1.000
2,92472145,9058,24.000,3.000
3,88026681,9030,22.000,2.000
4,98127795,9001,38.000,3.000


In [50]:
X.dtypes

individualnumber      int64
category_number       int64
hakkedis_amt        float64
odul_amt            float64
dtype: object

In [47]:
lgbm_model = LGBMRegressor(random_state=46)

In [57]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

lgbm_model = LGBMRegressor(random_state=46)
rmse = np.mean(
    np.sqrt(-cross_val_score(lgbm_model, X, y, cv=5, scoring="neg_mean_squared_error"))
)
rmse

0.12517676521431995

In [58]:
lgbm_params = {
    "learning_rate": [0.01, 0.03],
    "n_estimators": [500, 1000],
    "colsample_bytree": [0.3, 0.7],
}

lgbm_gs_best = GridSearchCV(lgbm_model, lgbm_params, cv=3, n_jobs=-1, verbose=True).fit(
    X, y
)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


In [60]:
lgbm_gs_best.best_params_
lgbm_final = lgbm_model.set_params(**lgbm_gs_best.best_params_, random_state=17).fit(
    X, y
)

In [61]:
rmse = np.mean(
    np.sqrt(-cross_val_score(lgbm_final, X, y, cv=5, scoring="neg_mean_squared_error"))
)

In [62]:
rmse

0.12333253909116812

In [92]:
y = test['cagri_count']
X = test.drop(["KESİNTİ_NO", "cagri_count","ind","İLÇE"], axis=1)
y_pred2 = lgbm_final.predict(X_test)

In [90]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13236 entries, 0 to 13235
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   individualnumber  13236 non-null  int64
 1   response          13236 non-null  int64
dtypes: int64(2)
memory usage: 206.9 KB


In [93]:
y_pred2[y_pred2 < 0] = 0
y_pred2[y_pred2 < 0.0005] = 1
y_pred2 = [round(pred) for pred in y_pred2]

In [96]:
sample_submission = pd.read_csv(
    "/kaggle/input/migros-datathon-coderspace/sample_submission.csv"
)

sample_submission["response"] = y_pred2

,individualnumber,response
0,95812936,0
1,23929570,0
2,95948115,0
3,100898513,0
4,12769156,0
5,17866909,1
6,96804798,0
7,94976682,0
8,93251259,0
9,100074640,0


In [83]:
sample_submission.dtypes

individualnumber    int64
response            int64
dtype: object

In [97]:
# you could use any filename. We choose submission here
sample_submission.to_csv("/kaggle/working/submission2.csv", index=False)

In [49]:
lgbm_final = lgbm_model.set_params(**lgbm_params).fit(X_train,y_train)

TypeError: '<=' not supported between instances of 'list' and 'int'